In [2]:
#Creating the functions
import pandas as pd

#load in the dataframes for functions
lemmaDF = pd.read_csv("Dictionary_Dataframes/greek_words.csv", sep = "\t")
del lemmaDF['Unnamed: 0']
#outputs list of matching IDs or an empty list
def get_id(word):
    #search the lemmaDF for the word
    output = []
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        if id not in output:
            output.append(row["id"])
    return(output)


#outouts dictionary of ids to the valid words 
def get_words_help(ids):
    output = {}
    if(type(ids) == list):
        for id in ids:
            curOut = []
            df = lemmaDF.loc[lemmaDF["id"] == id]
            for index, row in df.iterrows():
                #can add a checker for POS or add info about POS
                curOut.append(row["bare_text"])
            output[id] = curOut
        return output
       #for each id in the list, grab all the words with matching ID from parses and put in "outputs"
       #outputs is dict of ID --> parses with ID 
    else:
        print("Something went wrong...")
        return {}
    #enter FPP

def get_words(word):
    ids = get_id(word)
    return get_words_help(ids)
def check_enclitic(row, text):
    print("out of date")
def get_def(id):
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for index, row in df.iterrows():
        output = (row["definition"])
    return output
def get_lang(id):
    #returns ltn or grk
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for(index, row) in df.iterrows():
        output = row["lang_id"]
    if(output == 3 ):
        output = "ltn"
    else:
        output = "grk"
    return output
def get_all_ids():
    #returns a list of all noun ids
    output = []
    #for every id in lemmas, add it to output
    for index, row in lemmaDF.iterrows():
        out = row["id"]
        #if an int, add it to the output
        if(type(out) == type(1) and out not in output):
            output.append(out)
    return output

def make_dict_csv(ids, outputFile = "outputs/nounsDictionary.csv"):
    #ids is the list of ids for all words - check for nouns in here 
    FPP = []
    POS = []
    DICTENTRY = []
    SOURCE = []
    for id in ids:
        pos = "" 
        fpp = ""
        dictEntry = ""
        source = "morpheus"
        text = {}
        df = lemmaDF.loc[lemmaDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        #if no words in parses, we can't make an entry for it -- current outlook
        if(df.empty):
            continue
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"] 
                #hold the plural genative just in case
                if(morph[2] == "s" and (morph[7] == "g")):
                    if(text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                        text[6] = row["bare_text"] 
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "m")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]
                
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "f")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]

                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "n")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"] 
            #backup 4PP for verbs stored in text[6] - for weird words like futurus 
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
            
                if(morph) == "v1spia---":
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]                    
                if(morph) == "v--pna---":
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
                if(morph) == "v1sria---":
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
                if(morph) == "v3spia---":
                    if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                        text[4] = row["bare_text"]
                if(morph) == "v1spip---":
                    if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                        text[5] = row["bare_text"]
             #all other cases - simplest, just has 1 form 
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]  
        #set FPP to text[0]
        df2 = lemmaDF.loc[lemmaDF["id"] == id]
        if(pos == "noun"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            if(text[1] == "NULL"):
                text[1] = text[6]    
            dictEntry = text[0] + ", " + text[1] + ", " + text[2] 
            if(text[1] == "NULL"):
                dictEntry = text[0] + ", " + text[2] + " indecl."
            fpp = text[0]
            FPP.append(fpp)
            POS.append(pos)
            DICTENTRY.append(dictEntry)
        elif(pos == "verb"):
            #change which type by checking if we have a normal FPP
            if(text[0]!= "NULL"):
                kind = "regular"
            elif(text[5] != "NULL"):
                kind = "deponent"
            else:
                kind = "impersonal"
                #nested switch block! Yay!
            #fix edge case - 4PP is weird
            if(text[3] == "NULL"):
                    text[3] = text[6]
            match(kind):
                case "regular":
                    fpp = text[0]
                    dictEntry = text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                case "deponent":
                    fpp = text[5]
                    dictEntry = text[5] + ", " + text[1] + ", " + text[3]
                case "impersonal":
                    fpp = text[4]
                    dictEntry = text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            if(fpp == "NULL"):
                for index, row in df2.iterrows():
                    fpp = row["bare_text"]
            FPP.append(fpp)
            POS.append(pos)
            DICTENTRY.append(dictEntry)
        elif(pos == "adj"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + ", " + text[1] + ", " + text[2] 
            FPP.append(fpp)
            POS.append(pos)
            DICTENTRY.append(dictEntry)
        else:
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + " indecl."
            FPP.append(fpp)
            POS.append(pos)
            DICTENTRY.append(dictEntry)

    #end of function stuff
    out_map = {
        "FPP": FPP,
        "POS": POS,
        "Dictionary Entry": DICTENTRY,
    }
    print(len(FPP))
    print(len(POS))
    print(len(DICTENTRY))   
    output = pd.DataFrame(out_map)
    output.to_csv(outputFile, sep = "\t")

            


        
        
#function from word -> dictionary entry
def get_dict(word, show_def = False):
    ids = get_id(word)
        #list of dictionary entries 
    output = {}
    for id in ids:
        lang = get_lang(id)
        if(lang == "grk"):
            continue
        pos = "" 
        dictEntry = ""
        rowText = ""
        morph = ""
        text = {}
        definition = get_def(id)
        df = lemmaDF.loc[lemmaDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "n")):
                    #have to hard-code around que enclitic 
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):

                        text[0] = row["bare_text"]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
            
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "m")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]
                
                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "f")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]

                if((row["morph_code"][2] == "s")
                and (row["morph_code"][7] == "n") 
                and (row["morph_code"][6] == "n")
                and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"] 
            #backup 4th PP for verbs stored in text[6]
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if(morph) == "v1spia---":
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                        text[0] = row["bare_text"]                    
                if(morph) == "v--pna---":
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
                if(morph) == "v1sria---":
                    if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                        text[2] = row["bare_text"]
                if(morph) == "v3spia---":
                    if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                        text[4] = row["bare_text"]
                if(morph) == "v1spip---":
                    if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                        text[5] = row["bare_text"]                    
            #all other cases
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
        #preventing errors <3 
        if(type(definition) != type("str") or len(definition) == 0):
            definition = "No Definiton"
            #case for when FPP isn't found in parses
        if(text[0] == "NULL"):
            df2 = lemmaDF.loc[lemmaDF["id"] == id]
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
        #make dictionary entry string
        match(pos):
            #make sure each of those exists before trying to output
            case "noun":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ". - " + (definition)
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "adj":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + " - " + definition
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "verb":
                #change which type by checking if we have a normal FPP
                if(text[0]!= "NULL"):
                    kind = "regular"
                elif(text[5] != "NULL"):
                    kind = "deponent"
                else:
                    kind = "impersonal"
                    #nested switch block! Yay!
                #solve the edge case! 
                if(text[3] == "NULL"):
                    text[3] = text[6]
                match(kind):
                    case "regular":
                        if(show_def):
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                    case "deponent":
                        if(show_def):
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3]
                    case "impersonal":
                        if(show_def):
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            case other:
                dictEntry = pos + ": " + text[0]
                if(show_def):
                    dictEntry += " " + definition 
        output[id] = (dictEntry)
    return output

In [3]:
#Counting my errors 
dictionary_csv = pd.read_csv("outputs/Greek_dictionary.csv", sep = "\t")
del dictionary_csv["Unnamed: 0"]

def get_null():
    output = 0
    for index, row in dictionary_csv.iterrows():
        if("NULL" in row["Dictionary Entry"]):
            output += 1
    return output
def get_non_null():
    output = 0
    for index, row in dictionary_csv.iterrows():
        if("NULL" not in row["Dictionary Entry"]):
            output += 1
    return output
print("Number of NULL entries: " + str(get_null()))
print("Number of non-NULL entries: " + str(get_non_null())) 

Number of NULL entries: 16067
Number of non-NULL entries: 20098


Run block 1 to initialize functions in kernel

get_words(str word) - returns a list of all words with that input in its list of parses

get_dict(str word, bool print_def = False) - returns all dictionary entries for a given word. 
    Gives definition if print_def is true

make_noun_dict(int[] ids, str outputfile = default path) - create a CSV that is a dictionary of all nouns in the database

Current failure example: ID 33708

In [4]:
make_dict_csv(get_all_ids(), "outputs/Greek_dictionary.csv")

KeyboardInterrupt: 